!pip install conllu estnltk==1.6.7b0

!pip install pyxDamerauLevenshtein

Esmalt loeb failid listi ning viib nad paremini käideltatavale kujule

In [1]:
from estnltk import Text

sample_data = [
    ("Kersti Kaljuraid", [("2020", "2020.txt"), ("2019", "2019.txt"), ("2018", "2018.txt")]),
    ("Toomas Hendrik Ilves", [("2016", "2016.txt"), ("2015", "2015.txt"), ("2014", "2014.txt")]),
    ("Arnold Rüütel", [("2006", "2006.txt"), ("2005", "2005.txt"), ("2004", "2004.txt")]),
    ("Lennart Meri", [("2001", "2001.txt"), ("2000", "2000.txt"), ("1999", "1999.txt")])
]

def formatted_data(filename):
    return open(filename, "r", encoding="utf-8").read()

texts = [
    (
      name, 
      [(year, Text(formatted_data(filename)).tag_layer('morph_analysis')) for year, filename in filenames]
    ) for name, filenames in sample_data
]

teeb koopia tekstidest aga kus on sõnad asendatud sõna algvormidega

In [2]:
from pyxdameraulevenshtein import damerau_levenshtein_distance

lemma_cache = dict()  # estnltk has lazy fetching and thus getting lemmas can be cached
distances_cache = dict()

In [3]:
import re

def lemmafy_text(text):
    new_text = []
    for word in text.words:
        if not lemma_cache.get(word.text):
            lemma_cache[word.text] = word.lemma[0]
        new_text.append(lemma_cache[word.text])
    return Text(" ".join(new_text)).tag_layer('morph_analysis')

lemma_texts = [(name, [(year, lemmafy_text(text)) for (year, text) in president_texts]) for name, president_texts in texts]


def word_filter(word):
    return re.match(r"[^\s]+", word)
 

Kasutades varem eraldatud EV presidentide kõnesid, leia iga presidendi kohta üle kõigi tema kõnede kõige pikemad laused (Damerau-Levenshteini) kauguste 0, 1, 2 ja 3 kohta

In [4]:
distances = [0, 1, 2, 3]
variants = [("originaalseid", texts), ("algtüvestatuid", lemma_texts)]


def longest_sentence_given_distance_for_data(distance, data):
    sentences = [sentence for year, text in data for sentence in text.sentences]
    longest_length = 0
    longest = ("", "")
    
    for i, sentence1 in enumerate(sentences[0:-1]):
        for sentence2 in sentences[i + 1:]:
            a = [word.text for word in sentence1.words if word_filter(word.text)]
            b = [word.text for word in sentence2.words if word_filter(word.text)]
            aa = " ".join([word.text for word in sentence1.words])
            bb = " ".join([word.text for word in sentence2.words])
            
            if not distances_cache.get((aa, bb)):
                distances_cache[(aa, bb)] = damerau_levenshtein_distance(a, b)
            
            dist = distances_cache[(aa, bb)]
            
            if dist == distance and max(len(a), len(b)) > longest_length:
                longest_length = max(len(a), len(b))
                longest = (aa, bb)
    
    print(f"{longest[0]} - {longest[1]}") 


for description, variant in variants:
    print(f"Kõige pikemad laused kasutades {description} tekste\n")
    for name, president_data in variant:
        print(f"Presidendi {name} kõige pikemad laused kasutades")
        for distance in distances:
            print(f"kaugust {distance}: ", end="")
            longest_sentence_given_distance_for_data(distance, president_data)
        print()
    print("\n\n")

Kõige pikemad laused kasutades originaalseid tekste

Presidendi Kersti Kaljuraid kõige pikemad laused kasutades
kaugust 0: Kuidas me selleni jõudsime ? - Kuidas me selleni jõudsime ?
kaugust 1: Langes lund , langes külma lund . - Langes lund , langes valget lund .
kaugust 2: Palju õnne , armas Eesti ! - Palju õnne , armas eesti rahvas !
kaugust 3: Meie omadel ei ole . - Meie riik ei ole enam vaene .

Presidendi Toomas Hendrik Ilves kõige pikemad laused kasutades
kaugust 0: Mu daamid ja härrad . - Mu daamid ja härrad .
kaugust 1: Mu daamid ja härrad . - Mu daamid ja härrad ,
kaugust 2: teie hääl ongi riik . - Teie hääl ongi demokraatlik riik .
kaugust 3: Meie ei ole väikesed . - Need muutused ei ole kerged .

Presidendi Arnold Rüütel kõige pikemad laused kasutades
kaugust 0: Armas eesti rahvas , head kaasmaalased ! - Armas eesti rahvas , head kaasmaalased !
kaugust 1: Head kuulajad ! - Head iseseisvuspäeva !
kaugust 2: Head iseseisvuspäeva ! - Head iseseisvuspäeva meile kõigile !
kaugus

Võrreldes presitentide lauseid omavahel, leia iga presidenti kohta, milliste presidentidega on sel kauguste 0, 1, 2 ja 3 puhul kõige pikemad laused. 

In [5]:
distances = [0, 1, 2, 3]
variants = [("originaalseid", texts), ("algtüvestatuid", lemma_texts)]

def longest_sentence_given_distance_for_data_mixed(distance, data1, data2):
    longest_length = 0
    longest = ("", "")
    
    for sentence1 in data1.sentences:
        for sentence2 in data2.sentences:
            a = [word.text for word in sentence1.words if word_filter(word.text)]
            b = [word.text for word in sentence2.words if word_filter(word.text)]
            aa = " ".join([word.text for word in sentence1.words])
            bb = " ".join([word.text for word in sentence2.words])
            
            if not distances_cache.get((aa, bb)):
                distances_cache[(aa, bb)] = damerau_levenshtein_distance(a, b)
            
            dist = distances_cache[(aa, bb)]
            
            if dist == distance and max(len(a), len(b)) > longest_length:
                longest_length = max(len(a), len(b))
                longest = (aa, bb)
    
    print(f"{longest[0]} - {longest[1]}") 


for description, variant in variants:
    print(f"Kõige pikemad laused kasutades {description} tekste\n")
    for i, (name, president_data) in enumerate(variant[0:-1]):
        for year, text in president_data:
            for name2, president_data2 in variant[i + 1:]:
                for year2, text2 in president_data2:
                    print(f"Presidentide {name}({year}) ja {name2}({year2}) kõige pikemad laused kasutades")
                    for distance in distances:
                        print(f"kaugust {distance}: ", end="")
                        longest_sentence_given_distance_for_data_mixed(distance, text, text2)
                    print()
    print("\n\n")

Kõige pikemad laused kasutades originaalseid tekste

Presidentide Kersti Kaljuraid(2020) ja Toomas Hendrik Ilves(2016) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: Head sõbrad , - Head sõbrad .
kaugust 2: Meie omadel ei ole . - Meie ei ole väikesed .
kaugust 3: Meie ettevõtjad saavad hakkama . - Meie saame suureks olemisega hakkama .

Presidentide Kersti Kaljuraid(2020) ja Toomas Hendrik Ilves(2015) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: Head sõbrad , - Head sõbrad .
kaugust 2: See on võimatu . - Eesti on kaitstud .
kaugust 3: Nii ei tohi . - Et internetti ei olnud .

Presidentide Kersti Kaljuraid(2020) ja Toomas Hendrik Ilves(2014) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: Ei . - Lõpetuseks .
kaugust 2: See on võimatu . - See on igapäevane poliitika .
kaugust 3: See on võimatu . - See on meie enda huvides .

Presidentide Kersti Kaljuraid(2020) ja Arnold Rüütel(2006) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: Kal

Presidentide Toomas Hendrik Ilves(2016) ja Lennart Meri(1999) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: Ka Eestile . - Teiseks .
kaugust 3: Meie ei ole väikesed . - See ei ole eesmärk omaette .

Presidentide Toomas Hendrik Ilves(2015) ja Arnold Rüütel(2006) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: Eesti on kaitstud . - Eesti on olnud uuendusmeelne .
kaugust 3: See on meie tõde ja meie õigus . - See on meie eelis ja vastutus .

Presidentide Toomas Hendrik Ilves(2015) ja Arnold Rüütel(2005) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: Head sõbrad . - Head inimesed !
kaugust 3: Mu daamid ja härrad . - Ekstsellentsid , daamid ja härrad !

Presidentide Toomas Hendrik Ilves(2015) ja Arnold Rüütel(2004) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: Valige ! - Austatud kuulajad !
kaugust 3: Mu daamid ja härrad . - Ekstsellentsid , daamid ja härrad !

Presidentide Toomas Hendrik Ilv

Presidentide Kersti Kaljuraid(2019) ja Toomas Hendrik Ilves(2014) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: hoidma Eesti ! - elama Eesti !
kaugust 2: Armas Eesti rahvas ! - elama Eesti !
kaugust 3: aga kusagil olema ikkagi vari . - aga tema olema võimalik .

Presidentide Kersti Kaljuraid(2019) ja Arnold Rüütel(2006) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2: hea vabariik aastapäev ! - hea kuulaja !
kaugust 3: Eesti olema saama tundma lugu . - Eesti olema olnud uuendusmeelne .

Presidentide Kersti Kaljuraid(2019) ja Arnold Rüütel(2005) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: Armas Eesti rahvas ! - Armas rahvas !
kaugust 2: hea vabariik aastapäev ! - hea inimene !
kaugust 3: haigus olema paratamatu . - tegelikkus olema siiski teine .

Presidentide Kersti Kaljuraid(2019) ja Arnold Rüütel(2004) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: Armas Eesti rahvas ! - Armas rahvas !
kaugust 2: hoidma Eesti ! - austama kuulaj

Presidentide Toomas Hendrik Ilves(2014) ja Lennart Meri(2001) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: lõpetus . - esinema .
kaugust 2: see olema ulme . - eestlane olema võitma .
kaugust 3: aga mis toimuma kümme aasta pärast ? - aga täna kakskümmend aasta pärast ?

Presidentide Toomas Hendrik Ilves(2014) ja Lennart Meri(2000) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: lõpetus . - esimene .
kaugust 2: see muutus ei olema kerge . - see ei olema õige .
kaugust 3: see olema igapäevane poliitika . - see olema ainuõige majanduskasv suund .

Presidentide Toomas Hendrik Ilves(2014) ja Lennart Meri(1999) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1: lõpetus . - teine .
kaugust 2: see muutus ei olema kerge . - see valik ei olema tähtsusetu .
kaugust 3: see olema igapäevane poliitika . - see ei olema eesmärk omaette .

Presidentide Arnold Rüütel(2006) ja Lennart Meri(2001) kõige pikemad laused kasutades
kaugust 0:  - 
kaugust 1:  - 
kaugust 2:  - 
kaugust 

In [6]:
# Dynamic programming implementation of LCS problem 
# https://www.geeksforgeeks.org/printing-longest-common-subsequence/

# Returns length of LCS for X[0..m-1], Y[0..n-1]
def lcs(X, Y, m, n): 
    L = [[0 for x in range(n + 1)] for x in range(m + 1)] 
  
    # Following steps build L[m+1][n+1] in bottom up fashion. Note 
    # that L[i][j] contains length of LCS of X[0..i-1] and Y[0..j-1]  
    for i in range(m+1): 
        for j in range(n+1): 
            if i == 0 or j == 0: 
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]: 
                L[i][j] = L[i - 1][j - 1] + 1
            else: 
                L[i][j] = max(L[i - 1][j], L[i][j - 1]) 
  
    # Following code is used to print LCS 
    index = L[m][n] 
  
    # Create a character array to store the lcs string 
    lcs = [""] * (index + 1) 
    lcs[index] = "" 
  
    # Start from the right-most-bottom-most corner and 
    # one by one store characters in lcs[] 
    i = m 
    j = n 
    while i > 0 and j > 0: 
  
        # If current character in X[] and Y are same, then 
        # current character is part of LCS 
        if X[i - 1] == Y[j - 1]: 
            lcs[index - 1] = X[i - 1] 
            i -= 1
            j -= 1
            index -= 1
  
        # If not same, then find the larger of two and 
        # go in the direction of larger value 
        elif L[i - 1][j] > L[i][j - 1]: 
            i -= 1
        else: 
            j -= 1
  
    return " ".join(lcs)


def word_filter(word):
    return re.match(r"^[A-Za-zŠšÄäÕõÜüÖöŽž]*$", word)

Arvestades üht presidendi kõne ühe sõnade jadana leia iga presidendi kohta tema erinevad kõnesid aluseks võttes pikim ühisalamjada. (St algul on sisendiks esimese presidendi esimene ja teine kõne, seejärel esimene ja kolmas kõne jne. Väljundis on neli presidenti koos kahe aasta arvu ja pikima ühisalamjadaga.)

In [7]:
variants = [("originaalseid", texts), ("algtüvestatuid", lemma_texts)]

for description, variant in variants:
    print(f"Kõige pikemad ühis alamjadad {description} tekste\n")
    for name, president_data in variant:
        print(f"Presidendi {name} kõige pikem ühisalamjada")
        for i, (year, text) in enumerate(president_data[0:-1]):
            for year2, text2 in president_data[i + 1:]:
                a = [word.text for word in text.words if word_filter(word.text)]
                b = [word.text for word in text2.words if word_filter(word.text)]
                print(f"aastate {year} ja {year2} vahel: {lcs(a, b, len(a), len(b))}\n")
        print()
    print("\n\n")

Kõige pikemad ühis alamjadad originaalseid tekste

Presidendi Kersti Kaljuraid kõige pikem ühisalamjada
aastate 2020 ja 2019 vahel: Head Eesti seda mida ei on helgem Aga ka nii et kõigi kes on sest aga ka ja meil Me ja nende on Aga kogu ei ja selle kui on ja aastat ja on ei saa me ka et ei aga tegelikult väga ja ei oleks Jah on me meie siis ja on nii kui ka et meie ei küll on on ja eelkõige õigusriigi ja on meie riigid et Eesti Eesti ja ja töö ja meie meie tehakse julgeolekunõukogu on on mis on on ja ei seda seda See on meie See on ka on oluline ja on on ja et Meil palju kui Eesti ja nende ei ei ole meie mõni on et vaja see ei me aga kuigi ja ei saa ja see on eriti on et on häid tundub et nii ei need on on on ja aastal aga meie mis on on ja on ei saa on ja ja Eesti on ka on on on ja ja see täna on meie kõik see nende ei on kui et kui Eesti ja Hoiame Eestit 

aastate 2020 ja 2018 vahel: ja ja Eesti on ning oma et on me selle oleme oma ei et kui aga on meie ja ja on et et on see oli ja m


aastate 2001 ja 1999 vahel: Armsad kaasmaalased lugupeetavad külalised Eesti Vabariigi õnne Vabariigi sajandil tagasi ja ja ja kõnelda kui ja Eesti meie oma Euroopa ja Eesti Vabariigi on kui on meie ja oma aastapäeva oli nende meie ei vaid ja me Eesti ei ja ja juba juba Eesti mis ning ka ja et oma Eesti rahva aasta ja on Eesti See Eesti Eesti on on on et ja mis ei Eesti mis meie oma Eesti ühiskonna arengu või oma sõna kui kes on ja kes ega oma on muret mida see veel ei ja ei Eesti on Eesti ja ja ja kui ja Eesti et et ja Eesti ja me olla on ja on on ei tohi ja ja ja ja oma on Eesti Kuid on on on veel juba ja on on on riigi ja ja on ja Eesti kui on ka on on Eesti meie ja ja ja Eesti ja meie et me nagu ei ja Eesti ei Eesti on ja et või Eesti ei ole on ei on poliitikud ja Eesti on on mis ei ja Me ja selle ka ei ole on õigus on on ja seda on on on oma oma eest oma ja ühe ja Eesti Vabariigi 

aastate 2000 ja 1999 vahel: ja eest ja enne ja mis ja ja me täna kui ja meie Eesti on meie on kui o


aastate 2016 ja 2014 vahel: siin ja kodu Eesti ja maailm mina olema ja see nii kui kui see olema olema aeg kui olema ja see mina mina mis võima olema mina mina ka et Eesti olema mina saama mina olema olema olema olema ja ja ei olema olema mina olema Eesti olema see põhjus Eesti olema mina ei see see ka teine olema et ja vähem olema ja olema olema mis ise olema aeg olema see mina mina see olema ja mina ja mitte tulema olema nii inimene kui et mina ka ja olema inimene Eesti mina ja et aga ise olema see mina see inimene kes ja kes aga Eesti pere juurde olema rahvas see ja olema mina viima Eesti ei Euroopa see olema olema sama see kes mina mina ja olema mina kui mina pidama Eesti olema olema olema kui kes mina olema oma aga mis mina mis mina mina ja läinud tekitama iga ja olema mina kui Eesti see olema ja olema vastutus olema ka mis kuidas inimene pidama Eesti ja eest olema olema palju see olema aga see ei olema ei ja ja kui saama mina aga Eesti ja võimalus ja olema see mis olema ja olema


aastate 2001 ja 1999 vahel: armas kaasmaalane lugupeetav külaline sina kõik soovima Eesti vabariik aastapäev see sajand aasta tagasi ja ja ja mina kui Eesti ja olema Eesti mina olema see Eesti olema mis olema olema mina ja Eesti vabariik aastapäev et see olema olema ja mis kümme aasta oma mina aasta olema mina vaatama Eesti külm mis tema palju mina kuid mina ei mina ei mina ja mina olema Eesti mis Eesti tema olema ja olema Eesti Eesti ja et see Eesti rahvas mina see mis Euroopa olema see olema poliitik kes olema Eesti Eesti Eesti olema olema mina olema et tema ja mis ei Eesti mis mina olema et mina viima maailm Eesti olema ja kes olema kui olema mis Eesti või oma olema olema mina kui olema olema inimene ja see see olema kuidas see see poliitik Eesti see ei olema see ja ja tema ei ja olema olema olema ei tohtima olema eesti eesti olema Eesti mina olema olema olema Eesti see Eesti mina olema olema Eesti mitte ja mina olema mina olema olema kuid olema olema ikka veel mina mina tahtma ja 

Sama ülesanne nagu punktis 3, erinevusega, et iga presidendi kohta leitakse, millise presidendiga on sel kõige pikem ühisalamajada. (St algul on sisendiks esimese presidendi esineme kõne ja teise presidendi esineme kõne, seejärel esimese presidendi esimene kõne ja teise presidendi teine kõne jne. Väljundis on neli presidentide paari koos kahe aastaarvuga ja pikima ühisalamjadaga iga paari kohta.)

In [8]:
variants = [("originaalseid", texts), ("algtüvestatuid", lemma_texts)]

for description, variant in variants:
    print(f"Kõige pikemad laused kasutades {description} tekste\n")
    for i, (name, president_data) in enumerate(variant[0:-1]):
        for year, text in president_data:
            for name2, president_data2 in variant[i + 1:]:
                for year2, text2 in president_data2:
                    print(f"Presidentide {name}({year}) ja {name2}({year2}) kõige pikema ühisalamjada on:")
                    a = [word.text for word in text.words if word_filter(word.text)]
                    b = [word.text for word in text2.words if word_filter(word.text)]
                    print(f"{lcs(a, b, len(a), len(b))}\n")
    print("\n\n")


Kõige pikemad laused kasutades originaalseid tekste

Presidentide Kersti Kaljuraid(2020) ja Toomas Hendrik Ilves(2016) kõige pikema ühisalamjada on:
ja on ka ja ei ei saa on Aga ka seda on Tõsi ja ja oleme ja Mis et et sest ka see ei ja on meie ja ja on on see mis ei saa ega kus on ka et ja ja enda Euroopa tuleb ei me meie siis ja on ka kõik oma Aga on kui on ja meil ja ja ei on et Eesti Eesti kohustuse ja osaleda ja ja et ei on meie ja on et meie on ja me mis ja meile ei on meie on ja on et ja ning ei ole on ja on ja Kui saame meie ja Eesti ja on hoopis hakkama ei ei nagu on aga on et on et ei ikka ja ei me ja ja siis ei ja nii on see et riigi ei seda ka sest need on ja Eesti on Meie on Eesti on meil oma ja Eesti täna on on ju ja ja ja me me ja Eesti ja teha ja oleme Oleme on ja ja ja meie on on tööga et muutuvas maailmas siia on pole ja kus ei vaid meie 

Presidentide Kersti Kaljuraid(2020) ja Toomas Hendrik Ilves(2015) kõige pikema ühisalamjada on:
Head inimesed kodudes on aasta Aga


Presidentide Kersti Kaljuraid(2019) ja Toomas Hendrik Ilves(2014) kõige pikema ühisalamjada on:
Armas Eesti rahvas üle et on meile aastat ja on on Eesti inimesed kes on ja on on on on on ja ja mis ei ka võib on juurde on et ei ei ole neid me on ja ei aastat tagasi ja saab raha ja kas või on seista inimeste kui on ja ja ja oma mitte kui siis nende inimeste ja ei lihtsalt on ja on toime ja nagu ja on kes on on meie on on mida Meie kui ja on pole mida meil Eesti on Eesti ja on ja oma ka pole täna poolt ja on ja ja meie kui et me ei saa et ei Aga on Eesti ja on ja on on on See on mis mis on kui aastat inimesed ja nii on väga palju on ja Eesti on maailma või ja on aina ja on saab kui ka nii maailmas Eesti on enam ei ja See on on et oli sisse enne me ees see meie on Eesti Eesti on tänu ja on on kuidas seda saavutada Mul on hea meel et ja on kuidas et mis ei meie See ka on ja on on ja täna ei meie on See on ja ja et ei on kui veel Aga me ja 

Presidentide Kersti Kaljuraid(2019) ja Arnold Rüü


Presidentide Kersti Kaljuraid(2018) ja Arnold Rüütel(2006) kõige pikema ühisalamjada on:
ja ja rahva nimel ja Eesti Vabariigi meie vahel meie ja ja ja see on Eesti ja meie riik on ja ning ja meie ka Eesti meie meie ja ja ka ja Eesti eest ja on Eesti neile tuleb nii Eesti on ja on ja on on on ja meie mis on ja me oleme ja sellest kui üles palju ka ka ja on ja aga ka et olla ja Sama ja et ja ka ei kui ei siin et ei või on ja ja on et ja on ja ja ja seda ja ja on ja kui on see meie on ei ja kui ja võib küll aga ei ei seda mis oma seda ja ja ja ei ja oma või sõna eest ja ka oma ja ja kui ei või Aga ka ja on oma on vaid Eesti aasta pärast ja ja ja aasta pärast pole maailmas ja ja on on kui ka nende ja ja teadmine et on on siis me oma rahva ja ei vaid aga et Eesti Eesti et oma hea ja ka ja Eesti ja meie et ja riigi 

Presidentide Kersti Kaljuraid(2018) ja Arnold Rüütel(2005) kõige pikema ühisalamjada on:
meie ja ja Eesti ning ja meie Eesti Vabariigi on et on Eesti meie riigi kui maailmas on


Presidentide Toomas Hendrik Ilves(2016) ja Lennart Meri(2000) kõige pikema ühisalamjada on:
ja Eesti on ja on ei enam on et ei ka on ehk et meie ja oma ja ka meie oleme meie ja on ka ja ja see on meie ja ja on riik ja on ka Eesti mis ja ja Eesti ja Eesti ei on meie me ei ega aga ja on on kui ja on et on me on ja Eesti ja mida on on kui ja ja on meil ja ka tuleb ja ei mitte Eesti ja ka Eesti ja osaleda ja et selle ja kes on kelle Kui ja ja Eesti ja on et meie et Eesti on ja peame ja ja on Eesti on on Eesti on ei et Eesti ja Euroopa ja ja See ei ei ole ei ei võiks ja asemel aga ja ja üksnes ja on see ja kus ja nii on jääb ka et rohkem ja et on kui Eesti on on aga mitte ja ja Eesti ei ole ja on ja on ja ei ka ja see ei et riigi riigi Ja ei tohi et ka ja on ja on on ja on ja Eesti on see oma ja ning ja mis ja on ka ja ja ja ja Eesti Eesti ja ja on olnud ja midagi meie ja siin et on näen ja Siin on veel ja ja et oma Kui ja on ja Eesti ei et ja ja ja Eesti 

Presidentide Toomas Hendrik Ilve


Presidentide Toomas Hendrik Ilves(2014) ja Arnold Rüütel(2004) kõige pikema ühisalamjada on:
Armas rahvas ja ja ja ja võib ka ja ja ja kui nende on Eesti ja riigi ja ja ja ei ole ka aga ka on ei aga kui on et ja Eesti ja on on ja mida ja võivad saab ja meie ja on ja meie ja ega ka ja läbi ja ja sotsiaalne ja ja et Euroopa seda on kui ja Eesti on meie See Eesti ja peame ja ei ja ja kui kui on oma ja Eesti oma ja Eesti oma on See on on ja on Eesti ei rohkem ja on meie oma rohkem ja peame ja meid kui et ja rohkem ja ka et ei ole on ja meie seda on on ja on on Eesti enam ja ja on kui kunagi vajadus ja meie on olnud ja ja Eesti ja on ja ja ja ei on ja peame ja ja on ka et Eesti ja et selle on pole veel et Eesti ja meil kui et ning kui meie ka kes on meie üle on meil et Eesti ja et muutustega kui on ja seda ja 

Presidentide Toomas Hendrik Ilves(2014) ja Lennart Meri(2001) kõige pikema ühisalamjada on:
Eesti ja et me oma iseseisvuse aastat tagasi on Eesti on ja selles ja on ja Eesti on ning

...


Presidentide Toomas Hendrik Ilves(2016) ja Arnold Rüütel(2006) kõige pikema ühisalamjada on:
ja ja Eesti taas maailm ja mina nii ei see et mina olema ja võit mina rahvas ka aasta Eesti see olema mina Eesti saama Euroopa ja mina ja nagu mina saama ka Eesti ja Eesti ka mina mis ja ja Eesti riik ja rahvas ja olema et ja nii ka olema mina ja ja ja mis olema Euroopa Liidu olema kui mina ei mina pidama olema mis olema ja õigus mina ja see et ka olema see mitte ainult ka Eesti olema ja kohustus see pidama et mina ja vähendamine olema olema see samuti ja võima olema olema ja see ja olema siis tema mina pidama olema hea kuulaja oma väärtus ja ja pidama olema mina see ja teine ja ja kui ja olema olema see ise olema ei tema see oma kasutama see tähendama hea olema või see aga ei ega mis olema oma ja ja aga olema ei ja ega ei ja ja olema või oma ja ka pidama ja tegema olema riik ja see kui Eesti olema ja ja ka siis see Eesti olema olnud olema mina ja Eesti olema see et mina ja ja ja olema pärast 


Presidentide Toomas Hendrik Ilves(2015) ja Arnold Rüütel(2006) kõige pikema ühisalamjada on:
Eesti aasta ja ja mina see valik oma ja ja mina ja ka ei vaid see olema Eesti ja et mina rahu ja olema kogu mina päev ka see sõda olema ja maailm ja mina mina ka Euroopa mina olema ja olema olema tema kui Eesti olema olema ja olema Eesti Euroopa olema suurem kui olema ja et mina ei mina ei mina üks olema see olema olema NATO ka Eesti olema aga see ka mina ja ja ja olema palju see ka ei saama kui ei et olema vaid olema see viima olema just tema olema see muutuma ja nagu olema olema ja rahvas ja ja pidama olema mina ja ja ja ja olema olema mina olema et ei olema et oma hea ja olema hea ja ei ega olema riik oma kohustus ja ei ja ega ei olema või Eesti olema kui ka protsess mina see mina ja mis tegema olema oma ja mina Eesti olema ja olema olema olema mina ja pidama olema mina ja ja rohkem olema aasta olema aga nägema ja olema olema kui olema ja et kõik olema olema mina et just see rahvas mis ka m


Presidentide Toomas Hendrik Ilves(2014) ja Arnold Rüütel(2006) kõige pikema ühisalamjada on:
Armas Eesti rahvas ja ja ja kui mina mina iseseisvus see ja ja mina ja aasta üks poliitiline ja aasta võima et mina olema ja riik olema päev mina olema Eesti ja ja ja ka ka mina olema olema Eesti olema ei see kui ka Eesti olema ja olema ja ja mis aasta tagasi ja võima mina mure mina ja ei üks see ega vaid olema ja ja inimene kui ka olema mitte ainult vaid ka ja inimene tervis Eesti olema ja ise see pidama ei mina olema ja sama olema ja olema üldine ei kui olema olema see tulema ja inimene olema laps olema mina pidama see olema olema oma ja kuid see olema mina see vaid ja ja olema mis ja kui inimene ja olema vastutus olema mina vabadus olema olema see kasutama see ja see olema see kui ja aga ei saama ka ei suutma see mis olema tegema see ja ja olema iga ja kaks eest ei ja ja eest olema olema maailm protsess ja ja mis aasta töö tegema olema riik ja pidama see ja edu nii kui ka ei Eesti olema ka 


Presidentide Arnold Rüütel(2006) ja Lennart Meri(2001) kõige pikema ühisalamjada on:
Eesti Estonia Eesti vabariik mina käima mina taastama iseseisvus see ja ja mina rahvas ja see olema kõik Eesti vabariik aastapäev võima kinnitama et olema olema mina rahvas aasta mina see olema kuid mina Eesti mina mina ja mina olema ja ja olema ja mis olema ja see Eesti ja ja rahvas ja see mis olema Euroopa olema olema andma et mina meenutama et ja uus ei mina see olema see et avalik nii kui ka olema see ka ja Eesti olema tooma ning ei mina ja olema ja tulema ja tulema ja kui saama tegema olema olema ja see inimene või see et olema ja see ja pidama mina siis õppima olema ja ja olema olema mina ei olema see see ja olema ning see ja ka see olema kuid olema olema ei piir olema suur olema mina ja see ja ning olema oma ja kas või Eesti ja tema kui see olema see olema ja mina olema see mina ja ja olema olema Eesti mina ja tahtma mina rahvas Eesti kes olema ja olema ning ja olema leidma et see olema Eesti o

Presidentide Arnold Rüütel(2004) ja Lennart Meri(2001) kõige pikema ühisalamjada on:
kaasmaalane sina Eesti vabariik aastapäev Eesti riik mina iseseisvus ja ja inimene see olema mina ja Eesti ja kui riik ning see olema Eesti ja riik kindlustama riik ja see olema mis nagu olema mina leht mis vaid Eesti rahvas ja ja olema juba aasta ja olema ning et aga ja olema Eesti see olema Eesti olema kui Eesti aasta et ei suur Eesti mis võima areng nii kui ka Eesti see olema selge kui mina kui olema mina uus ja olema mina ei ja Eesti globaalne olema olema ja Eesti mina ja ja see olema mina tegema mina olema olema see Eesti ei ja ja oma ja ka kui eesmärk olema ja olema ja olema aeg olnud see olema see olema mis olema Eesti ja mina ka mina mis olema ja ja väärikas riik olema ka olema olema ei olema olema mina olema olema mina ja olema olema riik ning olema Eesti tulevik ja see ka ja see olema ja mina see olema mina ja ja et valima tee et mina see ei mina olema olema et Eesti ja ning ja ja olema et ei

Hinnang:

---

Lõppkokkuvõteks
- Damerau-Levenshteini algoritm oli päris kiire ja kasulik
- huvitaval kombel originaalsetel ning lemma kujul olevate tekstide Damerau-Levenshteini kauguste laused tulid sama pikad ning üleüldse sarnased. Sellest võib tekitada hüpoteesi, et kas tekstides kasutatavaid sõnade kombinisatsioone kasutati peamiselt samades käänetes?
- viimases kahes punktis on ootuspäraselt sõnad väga üldised ja mitte midagi ütlevad. Kuid osad fraasid korduvad seal.
- viimases kahes punktis lemma kujul olevad tekstid on tunduvalt pikemad